In [130]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path

In [131]:
PATH = Path("datasets")
EXT = "*.csv"

# dictionary holding all the files DataFrames with the format {"filename": file_DataFrame}
files_dfs = {}

# recursive search for csv files in PATH folder and subfolders 
for csv_file in PATH.rglob(EXT):
    filename = csv_file.name     # get the filename 
    df = pd.read_csv(csv_file)   # read the csv file as a DataFrame
    files_dfs[filename] = df     # add the DataFrame to the dictionary

In [132]:
for dataset in files_dfs:
    display(dataset)

'D_clients.csv'

'D_close_loan.csv'

'D_job.csv'

'D_last_credit.csv'

'D_loan.csv'

'D_pens.csv'

'D_salary.csv'

'D_target.csv'

'D_work.csv'

In [133]:
column_names = {}
res_list = []

for data in files_dfs:
    column_names.update({data:files_dfs[data].columns.tolist()})

for col in column_names.values():
    for col_name in col:
        res_list.append(col_name)


In [134]:
df_clients = pd.read_csv('datasets/D_clients.csv')
df_close_loan = pd.read_csv('datasets/D_close_loan.csv')
df_job = pd.read_csv('datasets/D_job.csv')
df_last_credit = pd.read_csv('datasets/D_last_credit.csv')
df_loan = pd.read_csv('datasets/D_loan.csv')
df_pens = pd.read_csv('datasets/D_pens.csv')
df_salary = pd.read_csv('datasets/D_salary.csv')
df_target = pd.read_csv('datasets/D_target.csv')
df_work = pd.read_csv('datasets/D_work.csv')

In [135]:
df_all_loans = df_loan.merge(df_close_loan, 'outer', 'ID_LOAN')
df_all_loans

,ID_LOAN,ID_CLIENT,CLOSED_FL
0,1753790658,106804370,1
1,1753790659,106804371,1
2,1753790660,106804372,1
3,1753790661,106804372,0
4,1753790662,106804373,1
...,...,...,...
21121,1753811779,106819588,1
21122,1753811780,106819589,0
21123,1753811781,106819590,0
21124,1753811782,106819591,0


In [136]:
len(df_all_loans['ID_CLIENT'].unique())

15223

In [137]:
counts = df_all_loans['ID_CLIENT'].value_counts() 
df_all_loans['LOAN_NUM_TOTAL'] = df_all_loans.groupby(['ID_CLIENT'])['ID_LOAN'].transform('count')
df_all_loans['LOAN_NUM_CLOSED'] =  df_all_loans.groupby(df_all_loans['ID_CLIENT'])['CLOSED_FL'].transform('sum')
#df_all_loans['LOAN_NUM_CLOSED'] = df_all_loans['LOAN_NUM_CLOSED'].fillna(0).astype(int)
df_all_loans

,ID_LOAN,ID_CLIENT,CLOSED_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,1753790658,106804370,1,1,1
1,1753790659,106804371,1,1,1
2,1753790660,106804372,1,2,1
3,1753790661,106804372,0,2,1
4,1753790662,106804373,1,1,1
...,...,...,...,...,...
21121,1753811779,106819588,1,3,2
21122,1753811780,106819589,0,1,0
21123,1753811781,106819590,0,1,0
21124,1753811782,106819591,0,1,0


In [138]:
df_all_loans['LOAN_NUM_TOTAL'].unique()

array([ 1,  2,  3,  4,  5,  6, 11,  7,  8], dtype=int64)

In [139]:
df_all_loans['LOAN_NUM_CLOSED'].unique()

array([ 1,  2,  0,  3,  4,  5,  6, 11,  7,  8], dtype=int64)

In [140]:
 df_all_loans.groupby(df_all_loans['ID_CLIENT'])['CLOSED_FL'].sum()

ID_CLIENT
106804370    1
106804371    1
106804372    1
106804373    1
106804374    1
            ..
106819588    2
106819589    0
106819590    0
106819591    0
106819592    0
Name: CLOSED_FL, Length: 15223, dtype: int64

In [141]:
#df_loan = df_loan['ID_CLIENT'].value_counts().rename_axis('ID_CLIENT').reset_index(name='LOAN_NUM_TOTAL')
#df_close_loan = df_close_loan['ID_CLIENT'].value_counts().rename_axis('ID_CLIENT').reset_index(name='LOAN_NUM_CLOSED')

In [142]:
df_clients = df_clients.rename(columns={'ID':'ID_CLIENT'})
df = df_clients.merge(df_target, 'outer', 'ID_CLIENT')
df = df.merge(df_job, 'outer', 'ID_CLIENT')
df = df.merge(df_last_credit, 'outer', 'ID_CLIENT')
df = df.merge(df_all_loans[['LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'ID_CLIENT']], 'outer', 'ID_CLIENT')
df = df.merge(df_salary, 'outer', 'ID_CLIENT')

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22312 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                22312 non-null  int64  
 1   AGE                      22312 non-null  int64  
 2   GENDER                   22312 non-null  int64  
 3   EDUCATION                22312 non-null  object 
 4   MARITAL_STATUS           22312 non-null  object 
 5   CHILD_TOTAL              22312 non-null  int64  
 6   DEPENDANTS               22312 non-null  int64  
 7   SOCSTATUS_WORK_FL        22312 non-null  int64  
 8   SOCSTATUS_PENS_FL        22312 non-null  int64  
 9   REG_ADDRESS_PROVINCE     22312 non-null  object 
 10  FACT_ADDRESS_PROVINCE    22312 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  22312 non-null  object 
 12  FL_PRESENCE_FL           22312 non-null  int64  
 13  OWN_AUTO                 22312 non-null  int64  
 14  AGREEMENT_RK          

In [144]:
df = df.drop_duplicates(keep = 'first')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                16000 non-null  int64  
 1   AGE                      16000 non-null  int64  
 2   GENDER                   16000 non-null  int64  
 3   EDUCATION                16000 non-null  object 
 4   MARITAL_STATUS           16000 non-null  object 
 5   CHILD_TOTAL              16000 non-null  int64  
 6   DEPENDANTS               16000 non-null  int64  
 7   SOCSTATUS_WORK_FL        16000 non-null  int64  
 8   SOCSTATUS_PENS_FL        16000 non-null  int64  
 9   REG_ADDRESS_PROVINCE     16000 non-null  object 
 10  FACT_ADDRESS_PROVINCE    16000 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  16000 non-null  object 
 12  FL_PRESENCE_FL           16000 non-null  int64  
 13  OWN_AUTO                 16000 non-null  int64  
 14  AGREEMENT_RK             15

In [145]:
df.loc[(df['ID_CLIENT'] == 106804372)]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,GEN_TITLE,JOB_DIR,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FAMILY_INCOME,PERSONAL_INCOME
5244,106804372,52,1,Неполное среднее,Состою в браке,4,0,1,0,Иркутская область,...,Специалист,Участие в основ. деятельности,84.0,33126.0,12.0,4000.0,2.0,1.0,от 10000 до 20000 руб.,9000.0


In [146]:
df = df.loc[df['TARGET'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                15223 non-null  int64  
 1   AGE                      15223 non-null  int64  
 2   GENDER                   15223 non-null  int64  
 3   EDUCATION                15223 non-null  object 
 4   MARITAL_STATUS           15223 non-null  object 
 5   CHILD_TOTAL              15223 non-null  int64  
 6   DEPENDANTS               15223 non-null  int64  
 7   SOCSTATUS_WORK_FL        15223 non-null  int64  
 8   SOCSTATUS_PENS_FL        15223 non-null  int64  
 9   REG_ADDRESS_PROVINCE     15223 non-null  object 
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object 
 12  FL_PRESENCE_FL           15223 non-null  int64  
 13  OWN_AUTO                 15223 non-null  int64  
 14  AGREEMENT_RK             15

In [147]:
df = df.drop(['GEN_INDUSTRY', 'GEN_TITLE', 'JOB_DIR'], axis=1)
df['WORK_TIME'].fillna(df['WORK_TIME'].median(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                15223 non-null  int64  
 1   AGE                      15223 non-null  int64  
 2   GENDER                   15223 non-null  int64  
 3   EDUCATION                15223 non-null  object 
 4   MARITAL_STATUS           15223 non-null  object 
 5   CHILD_TOTAL              15223 non-null  int64  
 6   DEPENDANTS               15223 non-null  int64  
 7   SOCSTATUS_WORK_FL        15223 non-null  int64  
 8   SOCSTATUS_PENS_FL        15223 non-null  int64  
 9   REG_ADDRESS_PROVINCE     15223 non-null  object 
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object 
 12  FL_PRESENCE_FL           15223 non-null  int64  
 13  OWN_AUTO                 15223 non-null  int64  
 14  AGREEMENT_RK             15

### Необходимые столбцы:

- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
- AGE — возраст клиента;
- SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- CHILD_TOTAL — количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- PERSONAL_INCOME — личный доход клиента (в рублях);
- LOAN_NUM_TOTAL — количество ссуд клиента;
- LOAN_NUM_CLOSED — количество погашенных ссуд клиента.

In [148]:
df[['AGREEMENT_RK', 'TARGET', 'WORK_TIME', 'CREDIT', 
    'TERM', 'FST_PAYMENT', 'LOAN_NUM_TOTAL',
    'LOAN_NUM_CLOSED', 'PERSONAL_INCOME']] = df[['AGREEMENT_RK', 'TARGET', 
                              'WORK_TIME', 'CREDIT', 'TERM', 
                              'FST_PAYMENT', 'LOAN_NUM_TOTAL', 
                              'LOAN_NUM_CLOSED', 'PERSONAL_INCOME']].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID_CLIENT                15223 non-null  int64 
 1   AGE                      15223 non-null  int64 
 2   GENDER                   15223 non-null  int64 
 3   EDUCATION                15223 non-null  object
 4   MARITAL_STATUS           15223 non-null  object
 5   CHILD_TOTAL              15223 non-null  int64 
 6   DEPENDANTS               15223 non-null  int64 
 7   SOCSTATUS_WORK_FL        15223 non-null  int64 
 8   SOCSTATUS_PENS_FL        15223 non-null  int64 
 9   REG_ADDRESS_PROVINCE     15223 non-null  object
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object
 12  FL_PRESENCE_FL           15223 non-null  int64 
 13  OWN_AUTO                 15223 non-null  int64 
 14  AGREEMENT_RK             15223 non-null  in

In [149]:
df

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,AGREEMENT_RK,TARGET,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FAMILY_INCOME,PERSONAL_INCOME
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,...,60099204,1,3,5588,6,1000,1,0,от 20000 до 50000 руб.,25000
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,...,62244665,0,5,19498,12,0,1,0,от 10000 до 20000 руб.,10000
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,...,61050759,0,360,15470,3,15000,1,1,от 20000 до 50000 руб.,30000
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,...,62079659,0,3,13960,6,2500,1,0,от 20000 до 50000 руб.,25000
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,...,66583553,0,12,11890,6,8000,2,2,от 10000 до 20000 руб.,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,...,66751099,0,36,11750,4,1900,2,2,от 5000 до 10000 руб.,8000
22308,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,...,64562377,0,24,12350,6,1380,1,0,от 20000 до 50000 руб.,12000
22309,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,...,62236542,0,36,4915,10,2000,1,0,от 5000 до 10000 руб.,9000
22310,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,...,66739926,0,36,5860,3,2000,1,1,от 10000 до 20000 руб.,12000


In [150]:
df['EDUCATION'].value_counts()

EDUCATION
Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: count, dtype: int64

In [151]:
df['MARITAL_STATUS'].value_counts()

MARITAL_STATUS
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: count, dtype: int64

In [152]:
df['FAMILY_INCOME'].value_counts()

FAMILY_INCOME
от 10000 до 20000 руб.    7077
от 20000 до 50000 руб.    6082
от 5000 до 10000 руб.     1512
свыше 50000 руб.           492
до 5000 руб.                60
Name: count, dtype: int64

In [153]:
def education(val):
    mapping ={
        'Среднее специальное': 0, 'Среднее':1, 'Высшее':2, 'Неоконченное высшее':3,
       'Неполное среднее':4, 'Два и более высших образования':5, 'Ученая степень':6
    }
    return mapping[val]
def marital_status(val):
    mapping = {
        'Состою в браке': 0, 'Не состоял в браке': 1, 'Разведен(а)': 2, 'Вдовец/Вдова': 3,
       'Гражданский брак': 4
    }
    return mapping[val]

def income(val):
    mapping = {
        'от 10000 до 20000 руб.': 0, 'от 20000 до 50000 руб.': 1,
       'от 5000 до 10000 руб.': 2, 'свыше 50000 руб.': 3, 'до 5000 руб.': 4
    }
    return mapping[val]

In [154]:
#df['EDUCATION'] = df['EDUCATION'].apply(education)
#df['MARITAL_STATUS'] = df['MARITAL_STATUS'].apply(marital_status)
#df['FAMILY_INCOME'] = df['FAMILY_INCOME'].apply(income)

In [188]:
#df_encoded = pd.get_dummies(df, columns=['EDUCATION', 'MARITAL_STATUS', 'FAMILY_INCOME'])
#df_encoded

In [189]:
df.columns

Index(['ID_CLIENT', 'AGE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS',
       'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
       'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE',
       'POSTAL_ADDRESS_PROVINCE', 'FL_PRESENCE_FL', 'OWN_AUTO', 'AGREEMENT_RK',
       'TARGET', 'WORK_TIME', 'CREDIT', 'TERM', 'FST_PAYMENT',
       'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'FAMILY_INCOME',
       'PERSONAL_INCOME'],
      dtype='object')

In [157]:
df

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,AGREEMENT_RK,TARGET,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FAMILY_INCOME,PERSONAL_INCOME
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,...,60099204,1,3,5588,6,1000,1,0,от 20000 до 50000 руб.,25000
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,...,62244665,0,5,19498,12,0,1,0,от 10000 до 20000 руб.,10000
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,...,61050759,0,360,15470,3,15000,1,1,от 20000 до 50000 руб.,30000
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,...,62079659,0,3,13960,6,2500,1,0,от 20000 до 50000 руб.,25000
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,...,66583553,0,12,11890,6,8000,2,2,от 10000 до 20000 руб.,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,...,66751099,0,36,11750,4,1900,2,2,от 5000 до 10000 руб.,8000
22308,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,...,64562377,0,24,12350,6,1380,1,0,от 20000 до 50000 руб.,12000
22309,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,...,62236542,0,36,4915,10,2000,1,0,от 5000 до 10000 руб.,9000
22310,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,...,66739926,0,36,5860,3,2000,1,1,от 10000 до 20000 руб.,12000


In [158]:
df.to_csv('full_table.csv')

In [175]:
df.columns

Index(['ID_CLIENT', 'AGE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS',
       'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
       'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE',
       'POSTAL_ADDRESS_PROVINCE', 'FL_PRESENCE_FL', 'OWN_AUTO', 'AGREEMENT_RK',
       'TARGET', 'WORK_TIME', 'CREDIT', 'TERM', 'FST_PAYMENT',
       'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'FAMILY_INCOME',
       'PERSONAL_INCOME'],
      dtype='object')